In [ ]:
%matplotlib inline
## Preamble
from sympy import Symbol, symbols, pprint, Interval
from sympy import sin, cos, sqrt, pi, oo
from sympy import diff, solve, nsolve, apart, expand, factor, lambdify, simplify, Eq, re, im
from sympy import init_printing

from sympy.integrals import integrate
from sympy.physics.vector import ReferenceFrame, divergence, gradient, scalar_potential
from sympy.abc import epsilon, beta

import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve

In [ ]:
### Declare coefficients and scalar variables as SymPy symbols
n, n0 = symbols('n n0') # density
nc, ns, nt = symbols('nc ns nt') # density
A = symbols('A') # amplitude
q = symbols('q', positive=True)        # reciprocal lattice spacing
R = ReferenceFrame('R') # coordinates: x=R[0], y=R[1]
init_printing()

## Setup density fields
nstr = n0 + A * cos(q*R[0])
ntri = n0 + A * (cos(q*R[1]/2)*cos(sqrt(3)*q*R[0]/2) - cos(q*R[1])/2)

In [ ]:
pprint(nstr)

In [ ]:
pprint(ntri)

In [ ]:
### Plot densities
## Prepare plotting arrays
N = 50

xs = np.zeros(N*N)
ys = np.zeros(N*N)
zs = np.zeros(N*N)
zt = np.zeros(N*N)

for j in range(N):
    for i in range(N):
        x = -10 + 20./N * i
        y = -10 + 20./N * j
        xs[N*j+i] = x
        ys[N*j+i] = y
        zs[N*j+i] = nstr.subs({n0:0, A:-0.2, q:1, R[0]:x, R[1]:y})
        zt[N*j+i] = ntri.subs({n0:0, A:-0.2, q:1, R[0]:x, R[1]:y})

In [ ]:
plt.figure()
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.title("Striped Phase")
Cs = plt.tricontourf(xs, ys, zs, cmap=plt.cm.get_cmap('coolwarm'))
plt.colorbar(Cs)

In [ ]:
plt.figure()
plt.xlabel("$x$")
plt.ylabel("$y$")
plt.title("Hexagonal Phase")
Ct = plt.tricontourf(xs, ys, zt, cmap=plt.cm.get_cmap('coolwarm'))
plt.colorbar(Ct)

In [ ]:
## Define free energy density

### Define density gradients
grad2str = divergence(gradient(nstr, R), R)
grad4str = divergence(gradient(divergence(gradient(nstr, R), R), R), R)

quadstr = nstr/2 * (epsilon*nstr + (nstr + 2*grad2str + grad4str))
cubestr = -beta/3 * nstr**3
quarstr = nstr**4 / 4

grad2tri = divergence(gradient(ntri, R), R)
grad4tri = divergence(gradient(divergence(gradient(ntri, R), R), R), R)

quadtri = ntri/2 * (epsilon*ntri + (ntri + 2*grad2tri + grad4tri))
cubetri = -beta/3 * ntri**3
quartri = ntri**4 / 4

In [ ]:
### Integrate energy over striped unit wavelength
astr = 2*pi/q
F_str = simplify(1/astr * expand(integrate(quadstr + cubestr + quarstr, (R[0], 0, astr))))
pprint(F_str)

In [ ]:
### Integrate energy over triangle unit cell
atri = 2*pi/q
F_tri = simplify(1/(2*atri*2/sqrt(3)*atri) * expand(integrate(quadtri + cubetri + quartri, (R[0], 0, 2*atri/sqrt(3)), (R[1], 0, 2*atri))))
pprint(F_tri)

In [ ]:
## Optimize Parameters

### Solve for q

qstr = solve(Eq(diff(F_str, q), 0), q)
pprint(qstr)

In [ ]:
qtri = solve(Eq(diff(F_tri, q), 0), q)
pprint(qtri)

In [ ]:
### Solve for A

Astr = solve(Eq(diff(F_str.subs({q:qstr[0]}), A), 0), A)
pprint(Astr)

In [ ]:
Atri = solve(Eq(diff(F_tri.subs({q:qtri[0]}), A), 0), A)
pprint(Atri)

In [ ]:
### Substitute optimal parameters into free energy expressions

F_con = F_tri.subs({q:1, A:0})
pprint(F_con)

In [ ]:
F_str = F_str.subs({q:qstr[0], A:Astr[2]})
pprint(F_str)

In [ ]:
F_tri = F_tri.subs({q:qtri[0], A:Atri[1]})
pprint(F_tri)

In [ ]:
## Plot free energy curves
bet = 0.5
eps = 0.05
xs = np.linspace(-0.5, 0.5, 100)
Fc = np.zeros(100)
Fs = np.zeros(100)
Ft = np.zeros(100)
for i in range(len(xs)):
    Fc[i] = re(F_con.subs({beta:bet, epsilon:eps, n0:xs[i]}))
    Fs[i] = re(F_str.subs({beta:bet, epsilon:eps, n0:xs[i]}))
    Ft[i] = re(F_tri.subs({beta:bet, epsilon:eps, n0:xs[i]}))

In [ ]:
plt.figure(figsize=(10,5))
plt.xlabel("$n_0$")
plt.ylabel("$F$")
plt.title(r"Free Energy $(\epsilon=0.05,\beta=0.5)$")
plt.plot(xs, Fc, label="constant")
plt.plot(xs, Fs, label="stripe")
plt.plot(xs, Ft, label="triangle")
plt.legend(loc="best")

In [ ]:
## Construct the Phase Diagram with $\beta=0$

### Define thermodynamic potentials

fcon = F_con.subs({beta:0, n0:nc})
dfcon = diff(fcon, nc)

fstr = F_str.subs({beta:0, n0:ns})
dfstr = diff(fstr, ns)

ftri = F_tri.subs({beta:0, n0:nt})
dftri = diff(ftri, nt)

In [ ]:
### Define equations to solve (constant and triangular phases)
eqn1 = dfcon - dftri
eqn2 = fcon - ftri - dfcon * (nc - nt)

### Generate numerically efficient expressions
variables = (nc, nt)
f1  = lambdify(variables, re(eqn1.subs({epsilon:-0.26})), modules='sympy')
f2  = lambdify(variables, re(eqn2.subs({epsilon:-0.26})), modules='sympy')

### Generate Jacobian matrix terms(for numeric solver)
j1c = lambdify(variables, re(diff(eqn1.subs({epsilon:-0.26}), nc)), modules='sympy')
j1t = lambdify(variables, re(diff(eqn1.subs({epsilon:-0.26}), nt)), modules='sympy')

j2c = lambdify(variables, re(diff(eqn2.subs({epsilon:-0.26}), nc)), modules='sympy')
j2t = lambdify(variables, re(diff(eqn2.subs({epsilon:-0.26}), nt)), modules='sympy')

In [ ]:
def eqncouple(N):
    return [f1(N[0],N[1]),
            f2(N[0],N[1])]

def eqnjacob(N):
    return [[j1c(N[0],N[1]), j1t(N[0],N[1])],
            [j2c(N[0],N[1]), j2t(N[0],N[1])]]

guess = [-0.35, -0.30]
solcontri = fsolve(func=eqncouple, x0=guess, fprime=eqnjacob)
print(solcontri)

In [ ]:
## Plot the tangent to confirm this result
xs = np.linspace(-0.35, -0.29, 100)
Fc = np.zeros(100)
Fs = np.zeros(100)
Ft = np.zeros(100)
for i in range(len(xs)):
    Fc[i] = re(F_con.subs({beta:0, epsilon:-0.26, n0:xs[i]}))
    Ft[i] = re(F_tri.subs({beta:0, epsilon:-0.26, n0:xs[i]}))

tangent = (re(F_con.subs({beta:0, epsilon:-0.26, n0:solcontri[0]})),
           re(F_tri.subs({beta:0, epsilon:-0.26, n0:solcontri[1]})))

plt.figure(figsize=(15,7))
plt.xlabel("$n_0$")
plt.ylabel("$F$")
plt.title(r"Free Energy $(\epsilon=-0.26,\beta=0.0)$")
plt.plot(xs, Fc, label="constant", zorder=10)
plt.plot(xs, Ft, label="triangle", zorder=10)
plt.plot(solcontri, tangent, label="tangent", color="black", zorder=1)
plt.legend(loc="best")

In [ ]:
## Construct the Phase Diagram

### Notice that the triangle phase falls in-between the constant and striped phases.